In [22]:
# 这个notebook 介绍了 如何对split learning 发起 inverse-model attack攻击

In [1]:
# 导包
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.attacks.model_inversion.inverse_model import InverseModelAttack
from ppsplit.utils.utils import create_dir
import torch
import os

# 防护措施
from ppsplit.defense.noise import Noise

# 模型、数据集获取
from target_model.task_select import get_dataloader_and_model, get_dataloader,get_models

In [2]:
# 超参数
args = {
        'device':torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        # 'dataset':'purchase',
        # 'dataset':'Iris',
        'model': 'ResNet18',
        # 'model': 'VGG5',
        'result_dir': 'inverse-model-results-20240414/',
        'oneData_bs': 30,
        'test_bs': 1,
        'train_bs': 32,
        'noise_scale':0, # 防护措施
        'split_layer': 7,
        'test_num': 'InverseModelAttack'
        # 'num_pairs': 10000, # RepE # 这个要另外准备
        }

print(args['device'])

cuda:1


In [ ]:
# 获取模型和数据集
# msg = get_dataloader_and_model(**args)

model_msg = get_models(args)

# one_data_loader,trainloader,testloader = model_msg['one_data_loader'],model_msg['trainloader'], model_msg['testloader']
client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
decoder_route = model_msg['decoder_route']
image_deprocess = model_msg['image_deprocess']

results_dir = model_msg['results_dir']
inverse_dir = results_dir + 'layer'+str(args['split_layer'])+'/'
data_type = 1 if args['dataset'] == 'CIFAR10' else 0

print('results_dir:',results_dir)
print('inverse_dir:',inverse_dir)
print('decoder_route:',decoder_route)

Num layers: 14
Split layer: 7
=> loading decoder model '../../results/inverse-model-results-20240414//Resnet18/InverseModelAttack//Decoder-layer7.pth'
results_dir: ../../results/inverse-model-results-20240414//Resnet18/InverseModelAttack/
inverse_dir: ../../results/inverse-model-results-20240414//Resnet18/InverseModelAttack/layer7/
decoder_route: ../../results/inverse-model-results-20240414//Resnet18/InverseModelAttack//Decoder-layer7.pth


In [ ]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_route,data_type=data_type,inverse_dir=inverse_dir)

# 加载decoder模型
if not os.path.isfile(decoder_route): # 如果没有训练decoder
    # 训练decoder
    args['train_bs']=32
    args['test_bs']=32
    msg_data = get_dataloader(args)
    # trainloader,testloader = get_cifar10_normalize(batch_size=32)
    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=msg_data['trainloader'],test_loader=msg_data['testloader'],
                            epochs=20)
else:
    print("Load decoder model from:",decoder_route)

print(decoder_net)

Load decoder model from: ../../results/inverse-model-results-20240414//Resnet18/InverseModelAttack//Decoder-layer7.pth
InversionNet(
  (layers): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): ConvTranspose2d(128, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  )
)


In [ ]:
# 实现攻击,恢复testloader中所有图片
# trainloader,testloader = get_cifar10_normalize(batch_size=1)
args['train_bs']=1
args['test_bs']=1
msg_data = get_dataloader(args)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=msg_data['trainloader'],test_loader=msg_data['testloader'],
                  deprocess=image_deprocess,
                  save_fake=True,
                  tab=msg_data['tabinfo'])

----train decoder----
client_net: 
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): GELU(approximate='none')
  (maxpool): AvgPool2d(kernel_size=3, stride=2, padding=1)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate='none')
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GEL

100%|██████████| 10000/10000 [05:05<00:00, 32.76it/s]

average euc: 2.8687425081729887
average mse: 0.29814175098091367
average ssim: 0.05805354984455571
average time: 0.01801928734779358 avg infer time:0.0014484070539474488


In [ ]:
# 查看中间输出的形状
image = iter(msg_data['testloader']).next()[0]
print("image.shape:",image.shape)
out = client_net(image)
print("out.shape:",out.shape)

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

# cifar10 （图像多分类）

In [6]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,get_one_data,deprocess

from target_model.models.VGG import VGG,VGG5Decoder,model_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 2 # 测试编号（对应结果文件夹名称）
split_layer = 2 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/BN+Tanh/VGG5-params-20ep.pth'
# unit_net_route = '/home/dengruijun/data/project/Inverse_efficacy/results/VGG5/BN+Tanh/2-20240101/VGG5-params-19ep.pth'
results_dir = f'../results/inverse-model-results-20240414/VGG5/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [7]:
# 准备基本模型client net
# split_layer_list = list(range(len(model_cfg['VGG5']))) # 可能的切割点

# 创建对应文件夹
create_dir(results_dir)
create_dir(inverse_dir)

# 把unit模型切割成client-server 的模型pair
client_net = VGG('Client','VGG5',split_layer,model_cfg,noise_scale=noise_scale)
pweights = torch.load(unit_net_route)
if split_layer < len(model_cfg['VGG5']):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)
print(client_net)


features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.running_var
features.1.num_batches_tracked
features.4.weight
features.4.bias
features.5.weight
features.5.bias
features.5.running_mean
features.5.running_var
features.5.num_batches_tracked
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
  )
  (denses): Sequential()
)


In [8]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=1,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = VGG5Decoder(split_layer=split_layer)
    # 训练decoder
    trainloader,testloader = get_cifar10_normalize(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)



=> loading decoder model '../results/inverse-model-results-20240414/VGG5/2/Decoder-layer2.pth'


In [18]:
# 实现攻击,恢复testloader中所有图片
trainloader,testloader = get_cifar10_normalize(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  deprocess=deprocess,
                  save_fake=True)

----train decoder----
client_net: 
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
  )
  (denses): Sequential()
)
decoder_net: 
VGG5Decoder(
  (features): Sequential(
    (0): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
  )
  (denses): Sequential()
)


100%|██████████| 10000/10000 [00:48<00:00, 204.41it/s]


average euc: 1.4440752688854932
average mse: 0.09039340291179251
average ssim: 0.8552792906537652
average time: 0.0020127411127090454 avg infer time:0.0003452569007873535


# Bank数据集 （表格数据二分类）

In [3]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_bank import preprocess_bank,tabinfo_bank

from target_model.models.BankNet import BankNet1,BankNetDecoder1,bank_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 2 # 测试编号（对应结果文件夹名称）
split_layer = 2 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型


# 重要路径设置
unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/Bank/bank-20ep_params.pth'
results_dir = f'../results/inverse-model-results-20240414/Bank/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [4]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = BankNet1(layer=split_layer,noise_scale=noise_scale)
pweights = torch.load(unit_net_route)
if split_layer < len(bank_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)



linear1.weight
linear1.bias
linear2.weight
linear2.bias


<All keys matched successfully>

In [5]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = BankNetDecoder1(layer=split_layer)
    # 训练decoder
    trainloader,testloader = preprocess_bank(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)

train decoder model...
[('D', 128, 64), 'LR', ('D', 63, 128)]
===============processing data===============
X_train.shape: (32950, 63)
X_test.shape: (8238, 63)
y_train.shape: (32950, 1)
y_test.shape: (8238, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
BankNet1(
  (linear1): Linear(in_features=63, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
)
decoder_net: 
BankNetDecoder1(
  (delinear1): Linear(in_features=64, out_features=128, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=128, out_features=63, bias=True)
)
Epoch 0


100%|██████████| 1030/1030 [00:08<00:00, 126.49it/s]


--- epoch: 0, train_loss: [63.452796936035156]
Epoch 1


100%|██████████| 1030/1030 [00:05<00:00, 189.63it/s]


--- epoch: 1, train_loss: [2.55757737159729]
Epoch 2


...

--- epoch: 17, train_loss: [0.16170364618301392]
Epoch 18


100%|██████████| 1030/1030 [00:05<00:00, 194.22it/s]


--- epoch: 18, train_loss: [0.17891734838485718]
Epoch 19


100%|██████████| 1030/1030 [00:05<00:00, 195.94it/s]

--- epoch: 19, train_loss: [1.9602798223495483]
model saved


In [6]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_bank(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True,tab=tabinfo_bank)

===============processing data===============
X_train.shape: (32950, 63)
X_test.shape: (8238, 63)
y_train.shape: (32950, 1)
y_test.shape: (8238, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
BankNet1(
  (linear1): Linear(in_features=63, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
)
decoder_net: 
BankNetDecoder1(
  (delinear1): Linear(in_features=64, out_features=128, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=128, out_features=63, bias=True)
)


100%|██████████| 8238/8238 [00:27<00:00, 298.54it/s]


average cos: 0.2624422825339745
average euc: 9.768948042181458
average mse: 1.862984648346539
average acc: 0.21766205389657683
average time: 0.0002857756053223486 avg infer time:0.00034716115411145793


## VFL

### data+model

In [ ]:
# 导包
import sys
sys.path.append('/home/yangjirui/data/drj/PP-Split')
sys.path.append('/home/yangjirui/data/drj/PP-Split/ppsplit/')
from ppsplit.attacks.model_inversion.UIFV.UIFV_model_data import *
# from ppsplit.utils.utils import create_dir
# import torch
# import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tab = {
    'boolList': [i for i in range(0, 22)],
    'onehot': {
        'marital': [0, 1, 2, 3], 'default': [4, 5, 6],
        'loan': [7, 8, 9], 'month': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        'poutcome': [20, 21, 22]
    },
    'numList': [i for i in range(23, 28)]
}

In [ ]:
# 设置超参数

parser = argparse.ArgumentParser("TabRebuild")
parser.add_argument('--multiple', action='store_true', help='Whether to conduct multiple experiments')
parser.add_argument('--name', type=str, default='decoder-rebuild', help='experiment name')
parser.add_argument('--data_dir', default='/home/yangjirui/data/vfl-tab-reconstruction/dataset/bank/bank-additional/bank-additional-full.csv',
                    help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--lr', type=float, default=0.001, help='init learning rate')
parser.add_argument('--eps', type=float, default=1e-3)
parser.add_argument('--workers', type=int, default=6, help='num of workers')
parser.add_argument('--k', type=int, default=2, help='num of client')
parser.add_argument('--seed', type=int, default=1, help='random seed')
parser.add_argument('--save',  default='/data/yangjirui/vfl-tab-reconstruction/rebuild-data/bank/v5/',
                    help='location of the data corpus')
parser.add_argument('--decoder_mode',
                    default="/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/attack-model/model+data+.pth.tar",
                    help='location of the decoder mode')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
# parser.add_argument('--inverse_data_output', default="/home/yangjirui/paper-code/data/adult/2layer-noloss/base-random/inverse_data.csv",
#                     help='location of the data corpus')
parser.add_argument('--weight_decay', type=float, default=3e-5, help='weight decay')
parser.add_argument('--gamma', type=float, default=0.97, help='weight decay')
parser.add_argument('--epochs', type=int, default=120, help='num of epochs')
parser.add_argument('--base_mode', default='/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/best.pth.tar', type=str, metavar='PATH',
                    help='path to latest base mode (default: none)')
parser.add_argument('--grad_clip', type=float, default=5., help='gradient clipping')

args, unknown = parser.parse_known_args()

In [ ]:
# 开始攻击
train, test = preprocess(args.data_dir)

acc, onehot_acc, num_acc, similarity, euclidean_dist = rebuild(train_data=train, test_data=test, tab=tab,
                                                               device=device, args=args)

print(f"acc:{acc}, onehot_acc:{onehot_acc}, num_acc:{num_acc}, similarity:{similarity}, euclidean_dist:{euclidean_dist}")

===============processing data===============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null

100%|██████████| 515/515 [00:08<00:00, 60.34it/s]


acc: 0.9878375584322187
onehot_acc: 0.9970449029126213
num_acc: 0.9786302139518158
similarity 0.8835727652144364
euclidean_dist 1.3748186748004654
################################ save data ############################
[[0.9878375584322187 0.9970449029126213 0.9786302139518158
  0.8835727652144364 1.3748186748004654 False 'decoder-rebuild'
  '/home/yangjirui/data/vfl-tab-reconstruction/dataset/bank/bank-additional/bank-additional-full.csv'
  64 0.001 0.001 6 2 1
  '/data/yangjirui/vfl-tab-reconstruction/rebuild-data/bank/v5/'
  '/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/attack-model/model+data+.pth.tar'
  0.9 3e-05 0.97 120
  '/data/yangjirui/vfl-tab-reconstruction/model/bank/v5/best.pth.tar' 5.0
  12]]
acc:0.9878375584322187, onehot_acc:0.9970449029126213, num_acc:0.9786302139518158, similarity:0.8835727652144364, euclidean_dist:1.3748186748004654


# Credit 数据集 （表格数据二分类）

In [7]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_credit import preprocess_credit,tabinfo_credit

from target_model.models.CreditNet import CreditNet1,CreditNetDecoder1,credit_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 2 # 测试编号（对应结果文件夹名称）
split_layer = 3 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/credit/credit-20ep_params.pth'
results_dir = f'../results/inverse-model-results-20240414/Credit/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/' # 储存
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置

In [8]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = CreditNet1(layer=split_layer,noise_scale=noise_scale)
pweights = torch.load(unit_net_route)
if split_layer < len(credit_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)


linear1.weight
linear1.bias
batch_norm1.weight
batch_norm1.bias
batch_norm1.running_mean
batch_norm1.running_var
batch_norm1.num_batches_tracked
linear2.weight
linear2.bias


<All keys matched successfully>

In [9]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = CreditNetDecoder1(layer=split_layer)
    # optimizer = torch.optim.SGD(decoder_net.parameters(), 1e-3)

    # 训练decoder
    trainloader,testloader = preprocess_credit(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20,
                            # optimizer=optimizer
                            )

train decoder model...
[('D', 512, 128), ('LR',), ('BN', 512), ('D', 250, 512)]
===============processing data===============
X_train.shape: (246008, 250)
X_test.shape: (61503, 250)
y_train.shape: (246008, 1)
y_test.shape: (61503, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
CreditNet1(
  (linear1): Linear(in_features=250, out_features=512, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ReLU1): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
)
decoder_net: 
CreditNetDecoder1(
  (delinear1): Linear(in_features=128, out_features=512, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=512, out_features=250, bias=True)
)
Epoch 0


100%|██████████| 7688/7688 [00:41<00:00, 185.19it/s]


--- epoch: 0, train_loss: [1.4743270874023438]
Epoch 1


100%|██████████| 7688/7688 [00:43<00:00, 174.87it/s]


--- epoch: 1, train_loss: [2.492779016494751]
Epoch 2


...

--- epoch: 17, train_loss: [0.2651415467262268]
Epoch 18


100%|██████████| 7688/7688 [00:43<00:00, 175.76it/s]


--- epoch: 18, train_loss: [0.6400712132453918]
Epoch 19


100%|██████████| 7688/7688 [00:42<00:00, 179.88it/s]

--- epoch: 19, train_loss: [1.0420310497283936]
model saved


In [10]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_credit(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True,tab=tabinfo_credit)

===============processing data===============
X_train.shape: (246008, 250)
X_test.shape: (61503, 250)
y_train.shape: (246008, 1)
y_test.shape: (61503, 1) <class 'numpy.ndarray'>
===============processing data end===============
----train decoder----
client_net: 
CreditNet1(
  (linear1): Linear(in_features=250, out_features=512, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ReLU1): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
)
decoder_net: 
CreditNetDecoder1(
  (delinear1): Linear(in_features=128, out_features=512, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=512, out_features=250, bias=True)
)


100%|██████████| 61503/61503 [10:44<00:00, 95.40it/s] 


average cos: 0.3584266871135943
average euc: 19.05002367519168
average mse: 1.6026163726057512
average acc: 0.2982943921434727
average time: 0.0002927957076459463 avg infer time:0.0004771134156478203


# Purchase100 数据集 （表格数据多分类）

In [2]:
# 导包和超参数设置
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase,tabinfo_purchase

from target_model.models.PurchaseNet import PurchaseClassifier1,PurchaseDecoder1,purchase_cfg
from target_model.models.splitnn_utils import split_weights_client

test_num = 2 # 测试编号（对应结果文件夹名称）
split_layer = 3 # 模型切割点 （split point）在该层之前的层（含），作为client的模型，之后的层作为server的模型

# 重要路径设置
unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/Purchase100/Purchase_bestmodel_param.pth'
results_dir = f'../results/inverse-model-results-20240414/Purchase/{test_num}/'
inverse_dir = results_dir + 'layer'+str(split_layer)+'/'
decoder_net_route = results_dir + f'Decoder-layer{split_layer}.pth' # 攻击的decoder net存储位置


In [3]:
# 准备target model的 client net（对模型进行切割）
create_dir(results_dir)
create_dir(inverse_dir)

client_net = PurchaseClassifier1(layer=split_layer,noise_scale=noise_scale)
pweights = torch.load(unit_net_route)
if split_layer < len(purchase_cfg):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)



linear1.weight
linear1.bias
linear2.weight
linear2.bias


<All keys matched successfully>

In [4]:
# 准备inverse_model attack使用到的东西
# 创建Inverse Model Attack对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=0,inverse_dir=inverse_dir)

# 加载decoder模型
if os.path.isfile(decoder_net_route): # 如果已经训练好了
    print("=> loading decoder model '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else: # 如果没有
    print("train decoder model...")
    decoder_net = PurchaseDecoder1(layer=split_layer)
    # 训练decoder
    trainloader,testloader = preprocess_purchase(batch_size=32)

    decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                            train_loader=trainloader,test_loader=testloader,
                            epochs=20)

=> loading decoder model '../results/inverse-model-results-20240414/Purchase/2/Decoder-layer3.pth'


In [5]:
# 实现攻击,恢复testloader中所有表格数据行
trainloader,testloader = preprocess_purchase(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  save_fake=True,tab=tabinfo_purchase)

purchase100 dataset processing...
datset route: /home/dengruijun/data/FinTech/DATASET/kaggle-dataset/Purchase100//data.npz
original dataset shape:  (197324, 600)
After random selection, dataset shape:  (197324, 600)
After split between classifier and attack: 
training dataset shape:  (157859, 600)
testing dataset shape:  (39465, 600)
Data loading finished
----train decoder----
client_net: 
PurchaseClassifier1(
  (linear1): Linear(in_features=600, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (Tanh2): Tanh()
)
decoder_net: 
PurchaseDecoder1(
  (delinear1): Linear(in_features=512, out_features=1024, bias=True)
  (Tanh1): Tanh()
  (delinear2): Linear(in_features=1024, out_features=600, bias=True)
  (Tanh2): Tanh()
)


100%|██████████| 39465/39465 [32:26<00:00, 20.27it/s]


average cos: 0.6405844313075421
average euc: 11.187545064271033
average mse: 0.2091979069004007
average acc: 0.3298509649900756
average time: 0.00045916630894636665 avg infer time:0.0008126230395732837
